In [1]:
'''
Title: Pitcher Performance Table
Author: Sam LaFell
Date: 11/17/2022
Purpose: Evaluating Pitcher Information

Granularity: xBA by pitch for each pitcher
'''

# Import Libraries
import pandas as pd
pd.set_option('display.max_columns', 500)
import pybaseball as pb
import os
import numpy as np
import re
from datetime import datetime, timedelta

In [12]:
# We've gotten batter information let's now get Pitcher Information in this notebook

# Generate a lookup table to use in the Game Performances function
# atl_players = get_players.get_players('2018-05-13', 'ATL', min_abs_season=10)

# This is how we start the batters... so let's look at a specific game and get the players on the opposing team
game_day = '2018-05-13'
team1 = 'ATL'
# team2 = Unknown for the moment

# Get as Datetime
game_day_dt = datetime.strptime(game_day, '%Y-%m-%d')

# Statcast for Entire Day
from pybaseball import statcast
statcast_gameday = statcast(start_dt=game_day)


datetime.datetime(2018, 5, 13, 0, 0)

Maybe this isn't the right way to approach it. Get pitching matchups for weeks prior and attempt to understand the starter for the next games xBA and other stats.

In [20]:
# retrieve all players' pitching stats for the month of May, 2017 
from pybaseball import pitching_stats_range
data = pitching_stats_range(game_day, game_day)
data

/Users/samlafell/opt/miniconda3/envs/MLB_Betting/lib/python3.10/site-packages/pybaseball/league_pitching_stats.py:68: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  table = table.drop('',1)


,Name,Age,#days,Lev,Date,Tm,,Opp,G,GS,W,L,SV,IP,H,R,ER,BB,SO,HR,HBP,ERA,GSc,AB,2B,3B,IBB,GDP,SF,SB,CS,PO,BF,Pit,Str,StL,StS,GB/FB,LD,PU,WHIP,BAbip,SO9,SO/W
1,Matt Albers,35,1649,Maj-NL,"May 13, 2018",Milwaukee,@,Colorado,1,0,NaN,NaN,NaN,1.1,2,1,1,0,2,1,0,6.75,,6,0,0,0,0,0,0,0,0,6,28,0.68,0.14,0.14,0.75,0.00,0.00,1.500,0.333,13.5,NaN
2,Brett Anderson,30,1649,Maj-AL,"May 13, 2018",Oakland,@,New York,1,1,NaN,1.0,NaN,5.0,8,4,4,2,4,1,0,7.20,37,22,2,0,0,1,0,0,0,0,24,97,0.57,0.16,0.08,0.72,0.11,0.00,2.000,0.412,7.2,2.0
3,John Axford,35,1649,Maj-AL,"May 13, 2018",Toronto,,Boston,1,0,NaN,NaN,NaN,1.1,1,0,0,0,1,0,0,0.00,,5,0,0,0,0,0,0,0,0,5,16,0.63,0.25,0.06,0.33,0.33,0.00,0.750,0.250,6.8,NaN
4,Matt Barnes,28,1649,Maj-AL,"May 13, 2018",Boston,@,Toronto,1,0,NaN,NaN,NaN,1.0,2,0,0,0,1,0,0,0.00,,4,1,0,0,0,0,0,0,0,4,14,0.64,0.07,0.07,0.50,0.25,0.00,2.000,0.667,9.0,NaN
5,Kyle Barraclough,28,1649,Maj-NL,"May 13, 2018",Miami,,Atlanta,1,0,NaN,NaN,NaN,1.0,0,0,0,1,1,0,0,0.00,,3,0,0,0,0,0,1,0,0,4,15,0.53,0.27,0.00,0.00,1.00,0.00,1.000,0.000,9.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,Arodys Vizcaino,27,1649,Maj-NL,"May 13, 2018",Atlanta,@,Miami,1,0,NaN,NaN,1.0,1.0,1,1,1,0,1,1,0,9.00,,4,0,0,0,0,0,0,0,0,4,14,0.64,0.14,0.07,0.33,0.33,0.00,1.000,0.000,9.0,NaN
124,Adam Wainwright,36,1649,Maj-NL,"May 13, 2018",St. Louis,@,San Diego,1,1,NaN,1.0,NaN,2.1,3,2,2,6,3,0,0,7.71,40,9,0,0,0,1,0,0,0,0,15,79,0.48,0.20,0.03,0.50,0.33,0.17,3.857,0.500,11.6,0.5
125,Dan Winkler,28,1649,Maj-NL,"May 13, 2018",Atlanta,@,Miami,1,0,NaN,NaN,NaN,1.0,0,0,0,0,2,0,0,0.00,,3,0,0,0,0,0,0,0,0,3,13,0.69,0.15,0.38,1.00,0.00,0.00,0.000,0.000,18.0,NaN
126,Nick Wittgren,27,1649,Maj-NL,"May 13, 2018",Miami,,Atlanta,1,0,NaN,NaN,NaN,1.0,0,0,0,0,1,0,0,0.00,,3,0,0,0,0,0,0,0,0,3,11,0.73,0.00,0.09,0.00,0.50,0.00,0.000,0.000,9.0,NaN
